## Importing Stuff

In [3]:
import sys
sys.path.append('/kaggle/input/helper1/')

In [4]:
from utils import pd, prepareDataFrame, getTensorflowDataset, getTokenizer

## Loading Data

In [5]:
train, test = prepareDataFrame("/kaggle/input/ner-data/ner.csv")

In [6]:
tokenizer = getTokenizer(pd.concat([train, test]))

In [7]:
vocabSize = len(tokenizer.word_index) + 1
vocabSize

27943

In [26]:
getTensorflowDataset??

Signature:
getTensorflowDataset(
    df: pandas.core.frame.DataFrame,
    tokenizer: keras.src.legacy.preprocessing.text.Tokenizer,
    maxLen: int,
    totalClasses: int,
    fields=('text', 'labels'),
    batchSize=64,
)
Docstring: <no docstring>
Source:   
def getTensorflowDataset(df:pd.DataFrame,tokenizer:Tokenizer,maxLen:int, totalClasses:int, fields=("text", "labels"), batchSize=64):
    dataset = tf.data.Dataset.from_generator(lambda : dataGenerator(df, tokenizer, maxLen, totalClasses, fields), output_signature=(
        tf.TensorSpec(shape=(maxLen,), dtype=tf.float32),
        tf.TensorSpec(shape=(maxLen, totalClasses), dtype=tf.float32),
    )).batch(batchSize)
    return dataset
File:      /kaggle/input/helper1/utils.py
Type:      function

In [45]:
trainingSet = getTensorflowDataset(train, tokenizer, 35, 18, batchSize=512)
testingSet = getTensorflowDataset(test, tokenizer, 35, 18, batchSize=512)

In [28]:
for x, y in testingSet:
    print(x.shape, y.shape)
    break

(512, 35) (512, 35, 18)


## Defining Model

In [19]:
from keras.layers import Bidirectional, LSTM, Dense, Input, Embedding, TimeDistributed, Dropout
from keras.models import Model

In [20]:
totalTokens = vocabSize
totalTokens

27943

In [21]:
def getModel(totalTokens:int, maxLen:int, embeddingSize:int, numClasses:int, name="Bi"):
    inputLayer = Input((maxLen,), name="Input")
    embeddingLayer = Embedding(totalTokens, embeddingSize, name="Embedding")(inputLayer)
    # input and embedding 

    lstm = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1), name="LSTM1")(embeddingLayer)
    lstm = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1), name="LSTM2")(lstm)
    lstm = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1), name="LSTM3")(lstm)
    
    dropout = Dropout(0.5, name="50Dropout")(lstm)

    classifications = TimeDistributed(Dense(numClasses, activation="softmax"), name="Classifier")(dropout)
    
    return Model(inputLayer, classifications, name=name)

In [46]:
model = getModel(totalTokens, 35, 200, 18)

In [47]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [48]:
model.summary()

Model: "Bi"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 35)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Embedding (Embedding)           │ (None, 35, 200)        │     5,588,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM1 (Bidirectional)           │ (None, 35, 256)        │       336,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM2 (Bidirectional)           │ (None, 35, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM3 (Bidirectional)           │ (None, 35, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 50Dropout (Dropout)             │ (None, 35, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Classifier (TimeDistributed)    │ (None, 35, 18)         │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,718,602 (25.63 MB)

 Trainable params: 6,718,602 (25.63 MB)

 Non-trainable params: 0 (0.00 B)

## Training Model

In [49]:
history = model.fit(trainingSet, epochs=10, validation_data=testingSet)

Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 863ms/step - accuracy: 0.7724 - loss: 1.0006 - val_accuracy: 0.9016 - val_loss: 0.4359
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 833ms/step - accuracy: 0.9078 - loss: 0.3728 - val_accuracy: 0.9382 - val_loss: 0.2066
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 837ms/step - accuracy: 0.9421 - loss: 0.1978 - val_accuracy: 0.9605 - val_loss: 0.1431
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 839ms/step - accuracy: 0.9624 - loss: 0.1377 - val_accuracy: 0.9668 - val_loss: 0.1185
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 836ms/step - accuracy: 0.9699 - loss: 0.1100 - val_accuracy: 0.9705 - val_loss: 0.1082
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 837ms/step - accuracy: 0.9742 - loss: 0.0943 - val_accuracy: 0.9718 - val_loss: 0.1034
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 835ms/step - accuracy: 0.9770 - loss: 0.0832 - val_accuracy: 0.9719 - val_loss: 0.1016
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 67s 835ms/step - accuracy: 0.9788 - loss: 0.0762 - val_accu

In [50]:
model.save("model.h5")

In [51]:
import pickle

In [52]:
with open("tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)

In [53]:
from utils import getEncoder

In [55]:
with open("encoder.pickle", "wb") as f:
    pickle.dump(getEncoder(pd.concat([train, test])), f)